Matches claim types for the micro corpus (no gold labels to check validity)

In [2]:

import json
import pandas as pd
import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import TextClassificationPipeline
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler



In [3]:
def read_in_data(num):
    corpus_list=["cmv-hidey","usdeb","essay_1","micro_struc"]

    data = []

    print("Choosen corpus is:",corpus_list[num])


    with open("Data/"+corpus_list[num]+".json") as f:
        for line in f:
            data.append(json.loads(line))

    df = pd.DataFrame(data)
    
    if num ==3:
       
        df=df.drop("discourse",axis=1)
    
    return df

In [4]:
df=read_in_data(3)

Choosen corpus is: micro_struc


In [18]:

frame_list=[]
for ex in range(len(df)):
    role_list=[]
    unit_ids=[]
    #get a list of the role and the start and end of the text belonging to them
    for i in df.iloc[ex,4]["units"]:
        role_list.append(i["attributes"].get("role"))
        unit_ids.append(i["tokens"])  

    #make a list of the text
    text=[]

    for part_list in df.iloc[ex,3]:
        tokens=part_list["tokens"]
        for word in tokens:
            text.append(word["surface"])

    #match the text to the unit ids
    t=[]

    for i in unit_ids:
        
        text_list=[]
        
        for j in i:
            text_list.append(text[j])
            
        t.append( " ".join(text_list))

    #combine into one frame
    pd.Series(t)
    pd.Series(role_list)

    mic_frame=pd.DataFrame(t,role_list)
    mic_frame=mic_frame.rename(columns={0: "text"})


    frame_list.append(mic_frame)

big_frame=pd.concat(frame_list)
big_frame


,text
Claim,As a central airport Berlin Tegel is particula...
Premise,It is an asset for the economy .
Premise,The negative impact of air traffic however is ...
Premise,Air traffic is extremely environmentally unfri...
Premise,"Particularly in Tegel , the residents have bee..."
...,...
Claim,Rent prices should be limited by a cap when th...
Premise,"Landlords may want to earn as much as possible ,"
Premise,"and many , consistent with market principles ,..."
Premise,but that people with the same income suddenly ...


In [61]:
#load model and tokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=3,
                                                      #num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.load_state_dict(torch.load('BERT Models/finetuned_BERT_epoch_3_small_set.model', map_location=torch.device('cpu')))



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
label_list=[]

text_list = big_frame['text'].tolist()

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=None)

for i in text_list:
    res=pipe(i)
    flat_list = [item for sublist in res for item in sublist]
    result = {}
    for j in flat_list:
        result.update(dict([tuple(j.values())]))

    label_list.append(max(result, key=result.get))

NameError: name 'model' is not defined

In [84]:
big_frame["pred_label"]=label_list
big_frame["pred_label"].replace("LABEL_0","Value",inplace=True)
big_frame["pred_label"].replace("LABEL_1","Fact",inplace=True)
big_frame["pred_label"].replace("LABEL_2","Policy",inplace=True)

big_frame["Type"] = big_frame.index
big_frame=big_frame.reset_index()
big_frame

,index,text,pred_label,Type
0,Claim,As a central airport Berlin Tegel is particula...,Fact,Claim
1,Premise,It is an asset for the economy .,Fact,Premise
2,Premise,The negative impact of air traffic however is ...,Fact,Premise
3,Premise,Air traffic is extremely environmentally unfri...,Fact,Premise
4,Premise,"Particularly in Tegel , the residents have bee...",Fact,Premise
...,...,...,...,...
571,Claim,Rent prices should be limited by a cap when th...,Policy,Claim
572,Premise,"Landlords may want to earn as much as possible ,",Policy,Premise
573,Premise,"and many , consistent with market principles ,...",Fact,Premise
574,Premise,but that people with the same income suddenly ...,Fact,Premise


In [79]:
big_frame["pred_label"].value_counts()

Fact      484
Policy     53
Value      39
Name: pred_label, dtype: int64

In [94]:
claim_frame=big_frame.loc[big_frame["Type"]=="Claim"]
claim_frame["pred_label"].value_counts()



Fact      73
Policy    28
Value     11
Name: pred_label, dtype: int64